In [23]:
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from sklearn.metrics import f1_score
from kaggle_secrets import UserSecretsClient
from sklearn.model_selection import train_test_split

import wandb
import torch
import spacy
from spacy import displacy
nlp = spacy.blank('en')
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
from transformers import BertForSequenceClassification

from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [24]:
try:
    user_secrets = UserSecretsClient()
    secret_value_0 = user_secrets.get_secret("wandb_api")
    wandb.login(key=secret_value_0)
    anony=None
except:
    anony = "must"
    
wandb.init(project="PICO Evidence Summarization",name=f"15.PICO Evidence Summarization BioBert-Base-Cased-v1.2-Run 15")

In [25]:
df=pd.read_csv('../input/pico-medical-literatures-related-to-heart-disease/PICO_Tags_Heart_Disease_Semi_Processed.csv')
df

,Sentences,Tags
0,methods total 107 women clinical indication in...,Population/Problem (element P)
1,patients her2 hr positive mbc labc randomized ...,Intervention and Comparison (element I and C)
2,statistical significance defined p-value 0 05,Not Relevant to the Evidence (Label N)
3,exclusion criteria included nephrotic proteinu...,Population/Problem (element P)
4,radial artery cannulation performed summarise ...,Intervention and Comparison (element I and C)
...,...,...
2692,crying grimacing reduced 82% 65% control infan...,Outcome (element O)
2693,design setting tobacco exercise diet messages ...,Population/Problem (element P)
2694,twenty-four-h holter recordings two-channel re...,Intervention and Comparison (element I and C)
2695,patients assigned bmsc control restricted rand...,Intervention and Comparison (element I and C)


In [26]:
df['Tags'].value_counts()

Intervention and Comparison (element I and C)    760
Outcome (element O)                              653
Not Relevant to the Evidence (Label N)           648
Population/Problem (element P)                   636
Name: Tags, dtype: int64

In [27]:
evidence_labels = df.Tags.unique()

evidence_dict = {}
for index, evidence_labels in enumerate(evidence_labels):
    evidence_dict[evidence_labels] = index
evidence_dict

{'Population/Problem (element P)': 0,
 'Intervention and Comparison (element I and C)': 1,
 'Not Relevant to the Evidence (Label N)': 2,
 'Outcome (element O)': 3}

In [28]:
df['Tags Mapped'] = df.Tags.map(evidence_dict)
df

,Sentences,Tags,Tags Mapped
0,methods total 107 women clinical indication in...,Population/Problem (element P),0
1,patients her2 hr positive mbc labc randomized ...,Intervention and Comparison (element I and C),1
2,statistical significance defined p-value 0 05,Not Relevant to the Evidence (Label N),2
3,exclusion criteria included nephrotic proteinu...,Population/Problem (element P),0
4,radial artery cannulation performed summarise ...,Intervention and Comparison (element I and C),1
...,...,...,...
2692,crying grimacing reduced 82% 65% control infan...,Outcome (element O),3
2693,design setting tobacco exercise diet messages ...,Population/Problem (element P),0
2694,twenty-four-h holter recordings two-channel re...,Intervention and Comparison (element I and C),1
2695,patients assigned bmsc control restricted rand...,Intervention and Comparison (element I and C),1


In [29]:
X_train, X_val, y_train, y_val = train_test_split(df.index.values,\
                                                  df['Tags Mapped'].values,\
                                                  test_size=0.15,\
                                                  random_state=42,\
                                                  stratify=df['Tags Mapped'].values)

In [30]:
df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [31]:
df.groupby(['Tags', 'Tags Mapped', 'data_type']).count()

Sentences
Tags                                          Tags Mapped data_type           
Intervention and Comparison (element I and C) 1           train            646
                                                          val              114
Not Relevant to the Evidence (Label N)        2           train            551
                                                          val               97
Outcome (element O)                           3           train            555
                                                          val               98
Population/Problem (element P)                0           train            540
                                                          val               96

In [52]:
df.head()

,Sentences,Tags,Tags Mapped,data_type
0,methods total 107 women clinical indication in...,Population/Problem (element P),0,train
1,patients her2 hr positive mbc labc randomized ...,Intervention and Comparison (element I and C),1,val
2,statistical significance defined p-value 0 05,Not Relevant to the Evidence (Label N),2,train
3,exclusion criteria included nephrotic proteinu...,Population/Problem (element P),0,val
4,radial artery cannulation performed summarise ...,Intervention and Comparison (element I and C),1,train


In [32]:
tokenizer = BertTokenizer.from_pretrained('dmis-lab/biobert-base-cased-v1.2', do_lower_case=True,truncation=True)

In [33]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].Sentences.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    truncation=True,
    padding='longest', 
    max_length=512, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].Sentences.values, 
    add_special_tokens=True, 
    truncation=True,
    return_attention_mask=True, 
    padding='longest', 
    max_length=512, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
tags_train = torch.tensor(df[df.data_type=='train']['Tags Mapped'].values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
tags_val = torch.tensor(df[df.data_type=='val']['Tags Mapped'].values)

In [34]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, tags_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, tags_val)

In [35]:
print(len(dataset_train), len(dataset_val))

2292 405


In [36]:
model = BertForSequenceClassification.from_pretrained("dmis-lab/biobert-base-cased-v1.2",
                                                      num_labels=len(evidence_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at dmis-lab/biobert-base-cased-v1.2 were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification we

In [37]:
batch_size = 32 #Defining Batch Size on which model has to be trained

# Converting tensors Dataset to dataloaders so that model can be trained
dataloader_train = DataLoader(dataset_train,sampler=RandomSampler(dataset_train), batch_size=batch_size) 

dataloader_validation = DataLoader(dataset_val, sampler=SequentialSampler(dataset_val), batch_size=batch_size)

In [38]:
optimizer = AdamW(model.parameters(),lr=8e-7, eps=1e-8)

In [39]:
epochs = 16
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0,num_training_steps=len(dataloader_train)*epochs)

In [40]:
def f1_score_func(preds, tags):
    preds_flat = np.argmax(preds, axis=1).flatten()
    tags_flat = tags.flatten()
    return f1_score(tags_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, tags):
    evidence_dict_inverse = {v: k for k, v in evidence_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    tags_flat = tags.flatten()

    for tag in np.unique(tags_flat):
        y_preds = preds_flat[tags_flat==tag]
        y_true = tags_flat[tags_flat==tag]
        print(f'PICO Evidence Class: {evidence_dict_inverse[tag]}')
        print(f'Accuracy: {len(y_preds[y_preds==tag])}/{len(y_true)}\n')

In [45]:
seed_val = 199
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [46]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [43]:
!mkdir models

mkdir: cannot create directory ‘models’: File exists


In [47]:
loss_train_avg_list = []
val_loss_list = []
val_f1_list = []

for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
    torch.save(model.state_dict(), f'models/finetuned_BioBERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    loss_train_avg_list.append(loss_train_avg)
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    val_loss_list.append(val_loss)
    tqdm.write(f'F1 validation Score (Weighted): {val_f1}')
    val_f1_list.append(val_f1)
    wandb.log({"Training loss":loss_train_avg, "Validation loss":val_loss,"F1 validation Score (Weighted)":val_f1,})

  0%|          | 0/16 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/72 [00:00<?, ?it/s]


Epoch 1
Training loss: 1.453814880715476
Validation loss: 1.3967433801064124
F1 validation Score (Weighted): 0.18677101184252826


Epoch 2:   0%|          | 0/72 [00:00<?, ?it/s]


Epoch 2
Training loss: 1.361994938717948
Validation loss: 1.3069188503118663
F1 validation Score (Weighted): 0.3887268602072913


...


Epoch 14
Training loss: 0.901660270161099
Validation loss: 0.9111812252264756
F1 validation Score (Weighted): 0.7332986540578789


Epoch 15:   0%|          | 0/72 [00:00<?, ?it/s]


Epoch 15
Training loss: 0.9032123858729998
Validation loss: 0.9067799219718347
F1 validation Score (Weighted): 0.7404734643431539


Epoch 16:   0%|          | 0/72 [00:00<?, ?it/s]


Epoch 16
Training loss: 0.8935957112246089
Validation loss: 0.9047684165147635
F1 validation Score (Weighted): 0.7426759949217728


In [48]:
model.load_state_dict(torch.load('models/finetuned_BioBERT_epoch_8.model', map_location=torch.device('cuda')))

<All keys matched successfully>

In [49]:
wandb.finish()

F1 validation Score (Weighted),▁▄▅▅▆▆▆▆▇▇▇█████
Training loss,█▇▆▅▄▄▃▃▂▂▂▁▁▁▁▁
Validation loss,█▇▆▅▄▄▃▃▂▂▂▁▁▁▁▁
F1 validation Score (Weighted),0.74268
Training loss,0.8936
Validation loss,0.90477


In [50]:
_, predictions, true_vals = evaluate(dataloader_validation)

In [51]:
accuracy_per_class(predictions, true_vals)

PICO Evidence Class: Population/Problem (element P)
Accuracy: 64/96

PICO Evidence Class: Intervention and Comparison (element I and C)
Accuracy: 70/114

PICO Evidence Class: Not Relevant to the Evidence (Label N)
Accuracy: 39/97

PICO Evidence Class: Outcome (element O)
Accuracy: 81/98

